## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi </a>  2024/9/20

#### Currently, the top two contestants in the competition have changed their names to 'trust LB'.

#### So is LB really worthy of our trust? In this notebook, I will conduct experiments on this issue.


#### This experiment was learned from the just concluded <a href="https://www.kaggle.com/competitions/isic-2024-challenge/overview">ISIC2024 competition</a>, and the code is as follows:<a href="https://www.kaggle.com/code/richolson/isic-2024-shake-up-lb-overfitting-simulator">ISIC 2024 Shake Up / LB Overfitting Simulator</a>.


#### Hint:

- 1.Regarding this issue, if you have any other experiments that interest you, you can leave a message in the discussion section, and I will continuously improve this notebook.

- 2.If you find any errors in my code or have any better suggestions, please feel free to point them out.



# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Libraries</h1></span>

In [ ]:
import polars as pl#和pandas类似,但是处理大型数据集有更好的性能.
import pandas as pd#导入csv文件的库
import numpy as np#对矩阵进行科学计算的库
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">random sample test data</h1></span>

####  In the data introduction, it is mentioned that there are approximately 60000 test data. Here, we sample 60000 training data, and since each data comes from a different game, we sample based on the game.

In [ ]:
train=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
train=train.to_pandas()
print(f"len(train):{len(train)}")

#采样大约60000的数据,考虑一次采样1种游戏
unique_games=list(train['GameRulesetName'].unique())
np.random.shuffle(unique_games)
test=train[train['GameRulesetName'].isin(unique_games[:360])].sort_values(['GameRulesetName']).reset_index(drop=True)

print(f"len(test):{len(test)}")
test.head()

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">public private split</h1></span>

#### Since the sampling of the test data is already random, we will divide it directly according to 0.35.

In [ ]:
test_games=list(test['GameRulesetName'].unique())
split=list(test[test['GameRulesetName']==test_games[128]].index)[0]
print(f"public_private_split:{split/len(test)}")

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">public private distribution</h1></span>

#### We can observe that the distributions of LB, final, and train each have their own characteristics.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.kdeplot(train['utility_agent1'].values,label='train')
sns.kdeplot(test[:split]['utility_agent1'].values,label='LB')
sns.kdeplot(test[split:]['utility_agent1'].values,label='final')
# 显示图例
plt.legend()
plt.show()

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">public private score</h1></span>

#### Here, we increase the RMSE of LB to the desired size by adding noise to the target, and observe the final score at this point.

In [ ]:
test['pred']=test['utility_agent1']
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))
def find_final_score(test,split,LB):#测试数据test根据split划分LB和final,输入LB的分数,看看final_score为多少
    test_copy=test.copy()
    LB_score=RMSE(test_copy[:split]['utility_agent1'].values,test_copy[:split]['pred'].values)
    while LB_score<LB:
        test_copy['pred']=test_copy['pred']+np.random.normal(loc=0, scale=0.5*(LB-LB_score+0.05), size=len(test))
        test_copy['pred']=test_copy['pred'].clip(-1,1)
        LB_score=RMSE(test_copy[:split]['utility_agent1'].values,test_copy[:split]['pred'].values)
        final_score=RMSE(test_copy[split:]['utility_agent1'].values,test_copy[split:]['pred'].values)
    return final_score
LB=0.434
final_score=find_final_score(test,split,LB=LB)
print(f"LB_score:{LB},final_score:{final_score}")

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">public private Correlation</h1></span>

#### Here, through multiple experiments, we will examine the correlation between public rankings and final scores.

#### We can observe that final_stcore is slightly larger than LB, and there is still a clear correlation between them.

In [ ]:
LB=[num/1000 for num in range(430,440)]*20
final_scores=[]
for lb in LB:
    final_scores.append(find_final_score(test,split,LB=lb))
plt.xlim(0.425,0.455)
plt.ylim(0.425,0.455)
plt.scatter(LB,final_scores)
plt.show()

#### By calculating the Pearson correlation coefficient, it can also be found that there is a high correlation between LB and final_stcore.

In [ ]:
df=pd.DataFrame({'LB':LB,'final':final_scores})
df.corr()

#### Do you think we can trust LB?